# 佇列系統模擬 (HW4)

本筆記本根據 HW4 的要求，模擬一個 M/M/1 佇列系統，並比較兩種服務情境。第一部分著重於模擬單一 M/M/1 佇列並驗證其系統時間。第二部分則比較單一 M/M/1 佇列（服務速率加倍）與兩個並行的 M/M/1 佇列（原始服務速率）的顧客平均等待時間。

模擬方法採用離散事件模擬。顧客的抵達間隔時間與服務時間均假設服從指數分佈，並使用反轉換抽樣法（Inverse Transform Sampling）產生隨機樣本。我們將收集並分析顧客的系統時間與等待時間等關鍵指標。

## 第一部分：M/M/1 佇列系統模擬

**目標**：模擬 M/M/1 佇列，記錄每位顧客的系統時間（從進入系統到完成服務的時間），並驗證模擬得到的平均系統時間是否接近理論值 E[T] = 1/(μ-λ)。

**參數設定**：將測試不同的抵達率 λ，服務率 μ 將保持固定或按比例調整。

## 第二部分：兩種服務情境比較

**目標**：比較兩種情境下顧客的平均等待時間（Wq），以判斷何者對顧客較有利。

**情境 A**：單一 M/M/1 佇列，服務速率變為 2μ。
其理論平均等待時間 Wq = λ / (2μ * (2μ - λ))。

**情境 B**：設立兩個獨立的 M/M/1 佇列，每個佇列的服務速率維持 μ，總抵達率 λ 平均分配給兩個佇列（即每個佇列的抵達率為 λ/2）。
其理論平均等待時間 Wq = (λ/2) / (μ * (μ - λ/2))。

**參數設定**：將基於第一部分的 λ 與 μ 值進行調整。


---

## 模擬結果

[程式連結](https://github.com/AsherJingkongChen/queue-system/blob/main/queue_system.ipynb)


In [10]:
import numpy as np
from numpy.typing import ArrayLike
import polars as pl
from typing import Tuple, List, Dict, Any

ARRIVAL_RATES_TO_TEST: List[float] = [0.2, 0.5, 0.8]
BASE_SERVICE_RATE: float = 1.0
CUSTOMER_COUNT: int = 10000
SIMULATION_SEED: int = 42

def generate_exponential_samples(rate: float, size: int, seed_offset: int = 0) -> ArrayLike:
    rng = np.random.default_rng(SIMULATION_SEED + seed_offset)
    uniform_samples = rng.uniform(size=size)
    return -np.log(uniform_samples) / rate

def compute_time_statistics(duration_samples: ArrayLike) -> Tuple[float, float]:
    return np.mean(duration_samples), np.var(duration_samples)

def calculate_theoretical_mean_system_time(arrival_rate: float, service_rate: float) -> float:
    if service_rate <= arrival_rate:
        return float('inf')
    return 1 / (service_rate - arrival_rate)

def calculate_theoretical_mean_wait_time(arrival_rate: float, service_rate: float) -> float:
    if service_rate <= arrival_rate:
        return float('inf')
    return arrival_rate / (service_rate * (service_rate - arrival_rate))

def simulate_queue_performance(arrival_rate: float, service_rate: float, customer_target: int, seed_offset: int = 0) -> Tuple[ArrayLike, ArrayLike]:
    inter_arrival_intervals = generate_exponential_samples(arrival_rate, customer_target, seed_offset)
    service_times = generate_exponential_samples(service_rate, customer_target, seed_offset + customer_target) 

    arrival_timestamps = np.cumsum(inter_arrival_intervals)
    service_start_timestamps = np.zeros(customer_target)
    service_completion_timestamps = np.zeros(customer_target)

    server_available_at_time = 0.0

    for i in range(customer_target):
        current_arrival_time = arrival_timestamps[i]
        service_start_timestamps[i] = max(current_arrival_time, server_available_at_time)
        service_completion_timestamps[i] = service_start_timestamps[i] + service_times[i]
        server_available_at_time = service_completion_timestamps[i]

    system_times_collected = service_completion_timestamps - arrival_timestamps
    wait_times_collected = service_start_timestamps - arrival_timestamps
    return system_times_collected, wait_times_collected

if __name__ == "__main__":
    pl_config_settings = {
        "ascii_tables": True,
        "float_precision": 4,
        "fmt_str_lengths": 180,
        "tbl_cell_alignment": "RIGHT",
        "tbl_cols": 180,
        "tbl_hide_column_data_types": True,
        "tbl_hide_dataframe_shape": True,
        "tbl_rows": 180,
        "tbl_width_chars": 180,
    }

    with pl.Config(**pl_config_settings):
        simulation_run_data: List[Dict[str, Any]] = []

        for i, arr_rate in enumerate(ARRIVAL_RATES_TO_TEST):
            system_times, _ = simulate_queue_performance(arr_rate, BASE_SERVICE_RATE, CUSTOMER_COUNT, seed_offset=i*10)
            mean_sim, var_sim = compute_time_statistics(system_times)
            mean_theory = calculate_theoretical_mean_system_time(arr_rate, BASE_SERVICE_RATE)
            simulation_run_data.append({
                "analysis_type": "M/M/1 System Time",
                "arrival_rate_input": arr_rate,
                "service_rate_input": BASE_SERVICE_RATE,
                "simulated_mean_time": mean_sim,
                "theoretical_mean_time": mean_theory,
                # "simulated_variance_time": var_sim
            })
        
        system_time_df = pl.DataFrame([
            item for item in simulation_run_data if item["analysis_type"] == "M/M/1 System Time"
        ])
        print("1. M/M/1 系統時間結果:")
        print(system_time_df.drop("analysis_type"))

        for i, arr_rate in enumerate(ARRIVAL_RATES_TO_TEST):
            service_rate_A = 2 * BASE_SERVICE_RATE
            _, wait_times_A = simulate_queue_performance(arr_rate, service_rate_A, CUSTOMER_COUNT, seed_offset=100+i*10)
            mean_wait_A_sim, var_wait_A_sim = compute_time_statistics(wait_times_A)
            mean_wait_A_theory = calculate_theoretical_mean_wait_time(arr_rate, service_rate_A)
            simulation_run_data.append({
                "analysis_type": "Wait Time Comparison",
                "scenario": "A (1S,2μ)",
                "base_arrival_rate": arr_rate,
                "arrival_rate": arr_rate,
                "service_rate": service_rate_A,
                "simulated_mean_time": mean_wait_A_sim,
                "theoretical_mean_time": mean_wait_A_theory,
                # "simulated_variance_time": var_wait_A_sim
            })

            arrival_rate_B = arr_rate / 2
            service_rate_B = BASE_SERVICE_RATE
            _, wait_times_B = simulate_queue_performance(arrival_rate_B, service_rate_B, CUSTOMER_COUNT, seed_offset=200+i*10)
            mean_wait_B_sim, var_wait_B_sim = compute_time_statistics(wait_times_B)
            mean_wait_B_theory = calculate_theoretical_mean_wait_time(arrival_rate_B, service_rate_B)
            simulation_run_data.append({
                "analysis_type": "Wait Time Comparison",
                "scenario": "B (2S,μ)",
                "base_arrival_rate": arr_rate,
                "arrival_rate": arrival_rate_B,
                "service_rate": service_rate_B,
                "simulated_mean_time": mean_wait_B_sim,
                "theoretical_mean_time": mean_wait_B_theory,
                # "simulated_variance_time": var_wait_B_sim
            })

        wait_time_comparison_df = pl.DataFrame([
            item for item in simulation_run_data if item["analysis_type"] == "Wait Time Comparison"
        ])
        print("2. 等待時間情境比較結果:")
        print(wait_time_comparison_df.drop("analysis_type"))


1. M/M/1 系統時間結果:
+--------------------+--------------------+---------------------+-----------------------+
| arrival_rate_input | service_rate_input | simulated_mean_time | theoretical_mean_time |
+=======================================================================================+
|             0.2000 |             1.0000 |              1.2332 |                1.2500 |
|             0.5000 |             1.0000 |              2.0185 |                2.0000 |
|             0.8000 |             1.0000 |              4.9247 |                5.0000 |
+--------------------+--------------------+---------------------+-----------------------+
2. 等待時間情境比較結果:
+-----------+-------------------+--------------+--------------+---------------------+-----------------------+
|  scenario | base_arrival_rate | arrival_rate | service_rate | simulated_mean_time | theoretical_mean_time |
+===========================================================================================================+
| A (1S,


---

## 結論

本次模擬研究成功完成了兩項主要任務：

1.  **M/M/1 系統驗證**：透過離散事件模擬，我們對 M/M/1 佇列系統的平均系統時間進行了估計。模擬結果顯示，在不同的顧客抵達率 λ (0.2, 0.5, 0.8) 和固定的服務率 μ (1.0) 條件下，模擬得到的平均系統時間與理論公式 E[T] = 1/(μ-λ) 的計算結果高度一致。這驗證了所建構模擬模型的準確性。

2.  **服務情境比較**：我們比較了兩種服務資源配置策略對顧客平均等待時間的影響：
    *   情境 A：單一服務站，服務速率加倍 (2μ)。
    *   情境 B：兩個獨立的並行服務站，每個服務站速率為 μ，分擔總抵達率。
    在所有測試的基礎抵達率 λ (0.2, 0.5, 0.8) 下，模擬數據一致表明，**情境 A（單一高速服務站）能夠提供更短的顧客平均等待時間**。這意味著在總服務能力相同（均為 2μ 的潛在處理能力）的情況下，將資源集中以提升單一服務點的效率，比將資源分散到多個處理速率較低的服務點，對減少顧客等待時間更為有利。

綜上所述，本模擬不僅驗證了 M/M/1 佇列的基本理論，也為服務系統設計提供了一個有價值的洞見：在特定條件下，提升服務速度可能比增加服務通道數量是更優的策略。

In [7]:
%%html
<style>
    .jp-Cell.jp-Notebook-cell:nth-last-child(3) .jp-Cell-inputArea,
    .jp-Cell.jp-Notebook-cell:last-child {
        display: none;
    }

    @page {
        size: A4 portrait;
        margin: 5mm 0 !important;
    }

    :root {
        --jp-content-link-color: dodgerblue;
    }

    a code {
        color: var(--jp-content-link-color) !important;
    }

    body {
        margin: 0 !important;
    }

    code,
    pre {
        font-family: Monaco, monospace !important;
        font-size: 10px !important;
        font-weight: 400 !important;
        line-height: 1.35 !important;
    }

    img {
        max-width: 80% !important;
    }

    h1 {
        text-align: center !important;
    }

    h1,
    h2,
    h3,
    h4,
    h5,
    h6,
    strong {
        font-weight: 700 !important;
    }

    h1,
    hr {
        page-break-before: always;
    }

    hr {
        visibility: hidden;
    }

    pre {
        white-space: pre-wrap;
    }

    table,
    td,
    th,
    tr,
    tbody,
    thead,
    tfoot {
        page-break-inside: avoid !important;
    }

    .jp-RenderedHTMLCommon {
        font-family: Calibri, Verdana, sans-serif !important;
        font-size: 12px !important;
        font-weight: 400 !important;
        line-height: 1.35 !important;
    }

    .jp-RenderedHTMLCommon td,
    .jp-RenderedHTMLCommon th,
    .jp-RenderedHTMLCommon tr {
        border: 1px solid var(--md-grey-500);
    }

    .jp-RenderedHTMLCommon table {
        margin-left: 2em;
    }

    .jp-CodeCell {
        margin-bottom: 1.5em;
    }
</style>